In [ ]:
import os
os.remove('snorkel.db')
os.system('cp snorkel.db\ corpus snorkel.db');

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

import os, sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

### Extraction

In [ ]:
from snorkel.models import Corpus

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()
print "%s contains %d Documents" % (corpus, len(corpus))

In [ ]:
from snorkel.models import candidate_subclass

Part = candidate_subclass('Part', ['part'])

In [ ]:
from hardware_utils import OmniNgramsHardware

hardware_ngrams = OmniNgramsHardware(n_max=3, split_tokens=None)

In [ ]:
from hardware_utils import load_extended_parts_dict
from snorkel.matchers import DictionaryMatch

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
parts_dict = load_extended_parts_dict(filename)
print "Loaded %d part numbers." % len(parts_dict)

parts_matcher = DictionaryMatch(d=parts_dict, ignore_case=True)

In [ ]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part, [hardware_ngrams], [parts_matcher])

In [ ]:
%time candidates = ce.extract(corpus.documents, 'Hardware Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

### Saving Candidates

In [ ]:
session.add(candidates)
session.commit()

In [ ]:
import os
os.system('cp snorkel.db snorkel.db\ candidates');

### Reloading Candidates

In [ ]:
from snorkel import SnorkelSession
session = SnorkelSession()

import os, sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

import os
os.remove('snorkel.db')
os.system('cp snorkel.db\ candidates snorkel.db');

from snorkel.models import Corpus
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()

In [ ]:
from snorkel.models import CandidateSet

candidates = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Candidates').one()
print "%s contains %d Candidates" % (candidates, len(candidates))

### Error Analysis

In [ ]:
print len(candidates)
print candidates[0]

In [ ]:
# TEMP
from hardware_utils import entity_level_total_recall

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(tp, fp, fn) = entity_level_total_recall(candidates, filename, relation=False)

In [ ]:
for c in list(fn):
    print c

In [ ]:
print candidates[0].part.parent.document.name

In [ ]:
from hardware_utils import expand_part_range

for p in expand_part_range(":BC548/549/550"): print p

In [ ]:
"BC 856 A" in parts_dict

In [ ]:
# for c in candidates:
#     if c.part.parent.document.name == "SIEMS01215-1":
#         import pdb; pdb.set_trace()

In [ ]:
# for c in candidates:
#     if c.part.get_span() == 'BC858A':
#         import pdb; pdb.set_trace()

In [ ]:
# def get_doc(corpus, doc_name):
#     for doc in corpus.documents:
#         if doc.name.upper() == doc_name.upper():
#             return doc

In [ ]:
# for c in candidates[100:200]:
#     print c

In [ ]:
# doc = get_doc(corpus, 'Infineon-BC857SERIES_BC858SERIES_BC859SERIES_BC860SERIES-DS-v01_01-en')
# for phrase in doc.phrases:
#     if 'BC857CW' in phrase.words:
#         import pdb; pdb.set_trace()